In [33]:
import pandas as pd
import re

train_data = pd.read_csv("Train.csv", sep=',')
test_data = pd.read_csv("Test.csv", sep=',')

X_train = train_data.text
y_train = train_data.sentiment
X_test = test_data.text
X_test_id = test_data.id

In [34]:
import nltk
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()

def lemmatize(sentence):
    sentence = sentence.split()
    for i in range(len(sentence)):
        sentence[i] = lemmatizer.lemmatize(sentence[i])
    sentence = ' '.join(sentence)
    return sentence

def preprocess_data(str):
    str = str.lower()
    str = re.sub(r'http\S+', '', str)
    str = re.sub(r'[^a-zA-Z !?]', '', str)
    str = lemmatize(str)
    return str

X_train = [preprocess_data(el) for el in X_train]
X_test = [preprocess_data(el) for el in X_test]

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
BoW_vectorizer = CountVectorizer()

#Build the feature set (vocabulary) and vectorise the Train dataset using BoW
X_train_BoW = BoW_vectorizer.fit_transform(X_train)

#Use the feature set (vocabulary) from Train to vectorise the Test dataset 
X_test_BoW = BoW_vectorizer.transform(X_test)

print("Train feature space size (using BoW):",X_train_BoW.shape)
print("Test feature space size (using BoW):",X_test_BoW.shape)

Train feature space size (using BoW): (21802, 33595)
Test feature space size (using BoW): (6099, 33595)


In [36]:
from sklearn.ensemble import RandomForestClassifier


random_forest_model=RandomForestClassifier(n_estimators=100)
random_forest_model.fit(X_train_BoW, y_train)
y_pred3 = random_forest_model.predict(X_test_BoW)

data = {'id':X_test_id,'text':X_test,'sentiment':y_pred3}
predictions = pd.DataFrame(data=data)
(predictions.drop(['text'], axis=1)).to_csv('rfmodel.csv', index=False)